In [1]:
import httpx
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from telegram import Bot

# Enable asyncio in a Jupyter environment
nest_asyncio.apply()

# Configuration
BOT_TOKEN = '6593706311:AAGfep182h5A1hOifip4095pVEy4Rp7kZTs'  
CHAT_ID = '@roofpocazz'  
GEMINI_API_URL = 'https://api.gemini.com/v1/pubticker/'
symbols = ['btcusdt', 'ethusdt', 'xrpusd', 'ltcusd', 'solusd', 'linkusd']  # Gemini symbols are in lowercase
##'adausd', 'xmrusd'

In [2]:
async def get_current_price(symbol):
    async with httpx.AsyncClient() as client:
        url = f'{GEMINI_API_URL}{symbol}'
        response = await client.get(url)
        
        if response.status_code == 200:
            try:
                data = response.json()
                price = data['last']  # Last traded price
                return price
            except Exception as e:
                print('Error processing response:', e)
                print(response.content)  # Print the full response for troubleshooting
        else:
            print(f'Failed to fetch data for {symbol}. Status code: {response.status_code}')
            return None

In [3]:
async def send_telegram_message(message):
    bot = Bot(token=BOT_TOKEN)
    await bot.send_message(chat_id=CHAT_ID, text=message)

In [4]:
async def fetch_and_send_prices():
    gmt_plus_6_time = datetime.utcnow() + timedelta(hours=6)
    message = f'GEMINI: Prices at {gmt_plus_6_time.strftime("%Y-%m-%d %H:%M:%S")} (GMT+6):\n\n'
    
    for symbol in symbols:
        price = await get_current_price(symbol)
        if price is not None:
            formatted_price = f'{float(price):.2f}'
            # Gemini uses lowercase for symbols, converting them to uppercase for display
            display_symbol = symbol.upper()
            message += f'{display_symbol}: ${formatted_price}\n'
    
    await send_telegram_message(message)

In [5]:
# Run the asyncio event loop
async def main():
    await fetch_and_send_prices()

loop = asyncio.get_event_loop()
loop.run_until_complete(main())